In [2]:
import getpass
import sys
#import urllib.request as ur
import io
import pycurl

import stem
import stem.connection
import stem.process
from stem.control import Controller
from stem.util import term

ip_echo_url = 'http://ipecho.net/plain'
js_test_url = 'http://127.0.0.1:81/test-js.html'

In [ ]:
controller = Controller.from_port(port=9051)
pw = getpass.getpass("Controller password: ")
controller.authenticate(password=pw)
controller.close()

In [ ]:
SOCKS_PORT = 7000

def query(url):
  """
  Uses pycurl to fetch a site using the proxy on the SOCKS_PORT.
  """

  output = io.BytesIO()

  query = pycurl.Curl()
  query.setopt(pycurl.URL, url)
  query.setopt(pycurl.PROXY, 'localhost')
  query.setopt(pycurl.PROXYPORT, SOCKS_PORT)
  query.setopt(pycurl.PROXYTYPE, pycurl.PROXYTYPE_SOCKS5_HOSTNAME)
  query.setopt(pycurl.WRITEFUNCTION, output.write)

  try:
    query.perform()
    return output.getvalue()
  except pycurl.error as exc:
    return "Unable to reach %s (%s)" % (url, exc)


# Start an instance of Tor configured to only exit through Russia. This prints
# Tor's bootstrap information as it starts. Note that this likely will not
# work if you have another Tor instance running.

def print_bootstrap_lines(line):
  if "Bootstrapped " in line:
    print(term.format(line, term.Color.BLUE))


print(term.format("Starting Tor:\n", term.Attr.BOLD))

tor_process = stem.process.launch_tor_with_config(
  config = {
    'SocksPort': str(SOCKS_PORT),
    #'ExitNodes': '{de}',
  },
  init_msg_handler = print_bootstrap_lines,
)

print(term.format("\nChecking our endpoint:\n", term.Attr.BOLD))
print(term.format(query("https://www.atagar.com/echo.php"), term.Color.BLUE))

tor_process.kill()  # stops tor

Starting Tor:

Sep 14 16:43:30.000 [notice] Bootstrapped 0%: Starting
Sep 14 16:43:30.000 [notice] Bootstrapped 5%: Connecting to directory server
Sep 14 16:43:30.000 [notice] Bootstrapped 80%: Connecting to the Tor network
Sep 14 16:43:31.000 [notice] Bootstrapped 85%: Finishing handshake with first hop


In [ ]:
SOCKS_PORT = 7000

tor_process = stem.process.launch_tor_with_config(
  config = {
    'SocksPort': str(SOCKS_PORT),
    'ExitNodes': '{ru}',
  }
)

print('ok...')

tor_process.kill()


In [ ]:

response = ur.urlopen(ip_echo_url)
#response = ur.urlopen(js_test_url)

html = response.readlines()
print(html)